In [1]:
%load_ext autoreload

%autoreload 2
import os


In [2]:

from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
from transformers import T5Config
from transformers import T5ForConditionalGeneration
from transformers import T5Tokenizer
from transformers import set_seed


# In[3]:


from datetime import datetime
import argparse
import os
import sys
import numpy as np

sys.path.append("./hf_transformers/")



In [3]:
import torch

from data_reader import GetDataAsPython
from prepare_data import create_data
from prepare_data import create_dataset
from prepare_data import extract_warning_types
from utils import boolean_string
from utils import get_current_time


# In[34]:


import torch

from data_reader import GetDataAsPython
from prepare_data import create_data
from prepare_data import create_dataset
from prepare_data import extract_warning_types
from utils import boolean_string
from utils import get_current_time
import csv

start_all = datetime.now()

# In[6]:

In [4]:
local = True
base_model = 'training/t5-small_repo-based_21-01-2022_10-29-42/checkpoint-16440'

if local:
    storage_directory = './storage/'
    load_model = f'./{storage_directory}/{base_model}'
    batch_size = 16
else:
    storage_directory = '/scratch/arminz/'
    batch_size = 64
    load_model = f'/{storage_directory}/{base_model}'

# In[7]:


import random


# In[8]:


exec_number = random.randint(0, 1000)
exec_number

118

In [5]:

# parser = argparse.ArgumentParser()
# parser.add_argument("-r", "--repo", type=str, default='/data/all/data/oroinc/platform')
# parser.add_argument("-p", "--percent", type=float, default=1)

# args = parser.parse_args()
repo = "/data/all/data/elastic/kibana" #args.repo
sample_percent = 1#args.percent

print('start:', repo, sample_percent)

lr = 4e-3
ws = 300
wd = 0.4
print('best arguments', lr, wd, ws)




# In[35]:


name='curr'
name


# In[36]:


# Read and prepare data
data = GetDataAsPython(f"{storage_directory}/data_and_models/data/data_autofix_tracking_repo_specific_final.json")
data_eslint = GetDataAsPython(f"{storage_directory}/data_and_models/data/data_autofix_tracking_eslint_final.json")
data += data_eslint


len(data)

start: /data/all/data/elastic/kibana 1
best arguments 0.004 0.4 300


104804

In [6]:

all_warning_types = extract_warning_types(data)


# In[39]:


(train_inputs, train_labels, val_inputs, val_labels, test_inputs, test_labels, train_info, val_info, test_info, ) = \
    create_data(data, all_warning_types, include_warning=True, design='repo-based-included', select_repo=repo)


# In[40]:


tokenizer = T5Tokenizer.from_pretrained(load_model)


# In[41]:


len(train_inputs)


# In[42]:



# Create dataset required by pytorch
samples = int(sample_percent * len(train_inputs))
train_dataset = create_dataset(train_inputs[:samples], train_labels[:samples], tokenizer, pad_truncate=True, max_length=128)
val_dataset = create_dataset(val_inputs, val_labels, tokenizer, pad_truncate=True)

print(f'amount of data that is being used for fine-tuning (train) : {len(train_dataset)} == {samples} ({sample_percent})')
print(f'amount of data that is being used for fine-tuning (validation): {len(val_dataset)} (full)')
print(f'amount of data that will be probably being used for testing: {sum([len(x) for x in test_inputs.values()])} (full)')

# In[61]:

splitting by : repo-based-included
train size: 136
val size: 44
test size: 51
amount of data that is being used for fine-tuning (train) : 136 == 136 (1)
amount of data that is being used for fine-tuning (validation): 44 (full)
amount of data that will be probably being used for testing: 51 (full)


/home/armin/TFix/env/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:185: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


In [7]:
type(train_dataset)

prepare_data.BugFixDataset

In [8]:
# from transformers import AutoTokenizer, DataCollatorWithPadding

# checkpoint = "bert-base-uncased"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)


# def tokenize_function(example):
#     return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


# tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
from torch.utils.data import Sampler, SequentialSampler
from typing import Sized, Iterator
class MySampler(Sampler[int]):
    data_source: Sized
    
    def __init__(self, data_source: Sized) -> None:
        self.data_source = data_source
        self.priority =  np.zeros(len(data_source))
    def __iter__(self) -> Iterator[int]:
        return reversed(np.argsort(self.priority))
    def __len__(self) -> int:
        return len(self.data_source)

    def set_priority(self, priority):
        self.priority = priority
        
sampler = MySampler(train_dataset)
# list(MySampler([1, 2, 3, 10]))

In [10]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)#, collate_fn=data_collator)
eval_dataloader = DataLoader(val_dataset, batch_size)#, collate_fn=data_collator)

In [11]:
# train_dataloader.sampler.set_priority(list(range(len(train_dataset) - 1)) + [-5])
# for ind, batch in train_dataloader:
#     print(ind)
# {k: v.shape for k, v in batch.items()}

In [12]:
# list(sampler)

In [13]:

now = datetime.now()
full_name = f'{name}_{exec_number}_{repo.rsplit("/", 1)[1][-20:]}_{sample_percent}'
model_directory = f'{storage_directory}/tmp/finetuned/{full_name}'
model_directory

'./storage//tmp/finetuned/curr_118_kibana_1'

In [14]:
tokenizer = T5Tokenizer.from_pretrained(load_model)
model = T5ForConditionalGeneration.from_pretrained(load_model)
model.resize_token_embeddings(len(tokenizer))
model.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(32104, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32104, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dro

In [15]:
num_train_epochs = 30


In [16]:
# predictions = np.array(outputs[1].argmax(-1).to('cpu'))
# labels = np.array(batch['labels'].to('cpu'))
# np.sum(np.all(np.equal(predictions, labels), axis=1))

In [17]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=lr, weight_decay=wd)

In [18]:
from transformers import get_scheduler

num_training_steps = num_train_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=ws,
    num_training_steps=num_training_steps,
)
print(num_training_steps)


270


In [19]:
import copy

In [20]:
# for index, batch in train_dataloader:
#     batch = {k: v.to('cuda') for k, v in batch.items()}
#     print(index)#, batch)
#     break

In [21]:
# [value.item() for key, value in sorted(list(zip(list(sampler)[:16], outputs[1].max(-1).values.sum(1))))]

In [22]:
model.eval()
scores = []
for batch in train_dataloader:
    batch = {k: v.to('cuda') for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    scores += [item.item() for item in outputs[1].max(-1).values.mean(1).to('cpu')]
    
# print(f'epoch #{epoch} | loss: {loss:.2f}, accuracy : {all_corrects/ all_cnt:.3f}')

In [23]:
from torch.nn.functional import softmax

[item.item() for item in softmax(outputs[1], dim=-1).max(-1).values.prod(-1).to('cpu')]

[0.38046014308929443,
 0.7871943116188049,
 0.5865635871887207,
 0.0036719662602990866,
 0.2610129415988922,
 0.689932107925415,
 0.36043503880500793,
 0.00020400254288688302]

In [24]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_train_epochs * len(train_dataloader)))

model.train()
best_val_accuracy, best_val_loss = 0, 1
patience = 10
best_model = copy.deepcopy(model)
no_imp = 0

for epoch in range(num_train_epochs):
    model.eval()
    scores = []
    for batch in train_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
#         scores += [item.item() for item in outputs[1].max(-1).values.mean(1).to('cpu')]
        scores += [item.item() for item in softmax(outputs[1], dim=-1).max(-1).values.prod(-1).to('cpu')]
    new_priorities = [value for key, value in sorted(list(zip(list(sampler), scores)))]    
    sampler.set_priority(new_priorities)
    print(list(sampler))
    print('---')
    
    model.train()
    all_corrects, all_cnt = 0, 0
    for batch in train_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs[0]#outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        
        predictions = np.array(outputs[1].argmax(-1).to('cpu'))
        labels = np.array(batch['labels'].to('cpu'))
        corrects = np.sum(np.all(np.equal(predictions, labels), axis=1))
        
        all_cnt += len(batch['labels'])
        all_corrects += corrects
        
#     print(f'epoch #{epoch} | loss: {loss:.2f}, accuracy : {all_corrects/ all_cnt:.3f}')    
    
    val_corrects, val_cnt = 0, 0
    for batch in eval_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        val_loss = outputs[0]
        predictions = np.array(outputs[1].argmax(-1).to('cpu'))
        labels = np.array(batch['labels'].to('cpu'))
        corrects = np.sum(np.all(np.equal(predictions, labels), axis=1))
        val_cnt += len(batch['labels'])
        val_corrects += corrects
    val_accuracy = val_corrects/ val_cnt
    print(f'epoch #{epoch} | tr_loss:{loss:.2f} tr_acc:{all_corrects/all_cnt} val_loss: {val_loss:.2f}, val_accuracy: {val_accuracy:.3f}')    
    
    
    if  val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        no_imp = 0
        best_model = copy.deepcopy(model)
        best_epoch = epoch
    else:
        no_imp += 1
    if no_imp >= patience:
        print(f'terminating... using {best_epoch}')
        break
    
    
    


[111, 113, 101, 105, 85, 77, 74, 69, 79, 65, 62, 75, 64, 88, 87, 83, 84, 68, 73, 78, 70, 86, 106, 91, 89, 76, 80, 90, 58, 81, 72, 63, 82, 67, 125, 124, 56, 55, 53, 122, 107, 31, 66, 127, 128, 71, 54, 94, 134, 112, 35, 120, 109, 98, 121, 95, 33, 3, 7, 12, 15, 32, 123, 24, 9, 36, 8, 92, 108, 93, 37, 18, 99, 115, 38, 114, 100, 40, 97, 19, 110, 39, 130, 103, 20, 131, 17, 51, 59, 116, 14, 27, 26, 119, 104, 133, 29, 135, 2, 28, 13, 60, 132, 34, 102, 126, 5, 10, 117, 118, 21, 16, 23, 1, 57, 6, 50, 25, 30, 41, 11, 96, 46, 49, 45, 52, 42, 48, 22, 0, 4, 43, 129, 47, 61, 44]
---
epoch #0 | tr_loss:0.73 tr_acc:0.3602941176470588 val_loss: 0.12, val_accuracy: 0.455
[111, 113, 101, 105, 75, 88, 74, 79, 62, 85, 77, 87, 69, 90, 65, 89, 64, 83, 84, 78, 73, 80, 68, 70, 91, 86, 76, 81, 106, 58, 82, 63, 72, 67, 56, 55, 53, 122, 31, 66, 107, 127, 128, 71, 54, 124, 120, 94, 134, 33, 109, 35, 125, 123, 98, 7, 112, 15, 24, 95, 92, 9, 8, 37, 36, 121, 108, 18, 3, 99, 40, 12, 32, 38, 115, 100, 97, 114, 110, 19, 

epoch #12 | tr_loss:0.17 tr_acc:0.5441176470588235 val_loss: 0.25, val_accuracy: 0.318
terminating... using 2


In [25]:
all_corrects, all_cnt = 0, 0
best_model.eval()
for batch in eval_dataloader:
    batch = {k: v.to('cuda') for k, v in batch.items()}
    with torch.no_grad():
        outputs = best_model(**batch)
    loss = outputs[0]
    predictions = np.array(outputs[1].argmax(-1).to('cpu'))
    labels = np.array(batch['labels'].to('cpu'))
    corrects = np.sum(np.all(np.equal(predictions, labels), axis=1))

    all_cnt += len(batch['labels'])
    all_corrects += corrects
    
print(f'epoch #{epoch} | loss: {loss:.2f}, accuracy : {all_corrects/ all_cnt:.3f}')

epoch #12 | loss: 0.12, accuracy : 0.705


In [26]:
len(outputs)

4

In [27]:
# from sklearn.metrics import accuracy_score
# import numpy as np
# def compute_metrics(p):
#     target_max_length = 256
#     predictions, labels = p.predictions, p.label_ids
    
#     predictions = np.pad(predictions, ((0, 0), (0, target_max_length - predictions.shape[1])), mode="constant")
#     predictions = np.delete(predictions, 0, axis=1)
#     predictions = np.insert(predictions, target_max_length - 1, 0, axis=1)

    

#     labels = np.array(labels)
#     labels = np.pad(labels, ((0, 0), (0, target_max_length - labels.shape[1])), mode="constant")
#     labels = np.delete(labels, 0, axis=1)
#     labels = np.insert(labels, target_max_length - 1, 0, axis=1)
    

#     correct_counter = np.sum(np.all(np.equal(labels, predictions), axis=1))
#     return {'acc': int(correct_counter)}


In [28]:

training_args = Seq2SeqTrainingArguments(
    output_dir=model_directory,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=ws,
    weight_decay=wd,
    logging_dir=model_directory,
    logging_steps=100,
    do_eval=True,
    evaluation_strategy="epoch",
    learning_rate=lr,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=1,
    eval_accumulation_steps=1,  # set this lower, if testing or validation crashes
    disable_tqdm=False,
    predict_with_generate=True,  # never set this to false.
    seed=42,  # default value
)

from transformers import EarlyStoppingCallback


trainer = Seq2SeqTrainer(
    model=best_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    optimizers=[torch.optim.Adam(params=model.parameters(), lr=lr), None],
    tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
)

In [29]:
# from sklearn.metrics import accuracy_score
# import numpy as np
# def compute_metrics(p):
#     target_max_length = 256
#     predictions, labels = p.predictions, p.label_ids
    
#     predictions = np.pad(predictions, ((0, 0), (0, target_max_length - predictions.shape[1])), mode="constant")
#     predictions = np.delete(predictions, 0, axis=1)
#     predictions = np.insert(predictions, target_max_length - 1, 0, axis=1)

    

#     labels = np.array(labels)
#     labels = np.pad(labels, ((0, 0), (0, target_max_length - labels.shape[1])), mode="constant")
#     labels = np.delete(labels, 0, axis=1)
#     labels = np.insert(labels, target_max_length - 1, 0, axis=1)
    

#     correct_counter = np.sum(np.all(np.equal(labels, predictions), axis=1))
#     return {'acc': int(correct_counter)}


In [30]:
# from transformers import EarlyStoppingCallback

# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     optimizers=[torch.optim.Adam(params=model.parameters(), lr=lr), None],
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
# )

In [31]:

# start_training = datetime.now()

# trainer.train()

# end_training = datetime.now()



In [32]:

tuned_model_dir = f'{model_directory}/best'
# tuned_model_dir='/scratch/arminz/tmp/finetuned'
trainer.save_model(tuned_model_dir)


end_all = datetime.now()
# import csv
# with open('tuner_runtime.csv', 'a') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerow([name, repo, len(train_dataset), len(val_dataset), base_model, start_all, start_training, end_training, end_all])

# In[78]:


In [33]:
if local:
    from numba import cuda
    device = cuda.get_current_device()
    device.reset()
#

In [34]:
result = os.system(f'python hf_transformers/tfix_testing.py --load-model {tuned_model_dir} -bs 16 --model-name t5-small -d repo-based-included -r {repo}')
print(result)

/home/armin/TFix/env/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:185: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
17it [00:10,  1.82it/s]              

start time:  23:08:41
['no-invalid-this', 'no-throw-literal', 'no-new-wrappers', 'guard-for-in', 'no-new-object', 'comma-style', 'prefer-spread', 'no-caller', 'no-extra-bind', 'no-array-constructor', 'prefer-rest-params', 'generator-star-spacing', 'no-this-before-super', 'no-extend-native', 'no-undef', 'no-useless-escape', 'no-dupe-keys', 'no-console', 'no-constant-condition', 'no-duplicate-case', 'no-empty', 'no-extra-semi', 'no-redeclare', 'no-cond-assign', 'no-extra-boolean-cast', 'no-fallthrough', 'no-unreachable', 'valid-typeof', 'no-unsafe-finally', 'no-unused-vars', 'no-debugger', 'no-unsafe-negation', 'no-case-declarations', 'no-self-assign', 'no-process-exit', 'no-inner-declarations', 'for-direction', 'no-compare-neg-zero', 'no-sparse-arrays', 'no-func-assign', 'no-const-assign', 'no-global-assign', 'use-isnan', 'no-unused-labels', 'require-yield', 'getter-return', 'no-dupe-class-members', 'no-ex-assign', 'constructor-super', 'no-new-symbol', 'no-empty-pattern', 'no-class-assi

In [35]:
import shutil

shutil.rmtree(tuned_model_dir)
#